scrape articles 

In [1]:
import requests
from bs4 import BeautifulSoup
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}

url = 'https://scholar.google.com/scholar?hl=fr&as_sdt=0%2C5&q=university+cadi+ayyad&oq=university+cad'

response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.content, 'html.parser')


In [ ]:
for block in soup :
    print(block)
    print("-----------------------------------------------------")

In [3]:
articles_blocks = soup.findAll('div',{'class':'gs_r'})
len(articles_blocks)

11

**Essayer avec le 1ere article**

In [4]:
import re
article=articles_blocks[0]
#----------------------article_id----------------------------------------------
id=article.find('h3').find('a')['id']


#-------------le lien de l'article---------------------------------------------
document_link = article.find('h3').find('a')['href']



#-------------le titre de l'article--------------------------------------------
title_element = article.find('h3', {'class': 'gs_rt'}).text.strip()


#------------------les auteurs de l'article-------------------------------------
authors_data = article.find('div', class_='gs_a').findAll('a')
authors =[]
for author in authors_data:
    author_info={}
    author_info['name']=author.text
    author_info['bio']=author['href']
    authors.append(author_info) 
    

#--------------------le nombre de citations de l'article---------------------------
citation_element = article.find(lambda tag: tag.name == 'a' and 'Cité' in tag.text)
#-------------------le lien des citations-----------------------------------------
citation_link = citation_element['href']
citation_count=int(re.search(r'\d+', citation_element.text).group())


#------------------------le nombre de versions---------------------------------------
version_element=article.find(lambda tag: tag.name == 'a' and 'versions' in tag.text)
#------------------------le lien des versions------------------------------------------
version_link = version_element['href']
version_count= int(re.search(r'\d+', version_element.text).group())
#------------------------les autres article en relation-----------------------------
autres_article= soup.find(lambda tag: tag.name == 'a' and tag.text == 'Autres articles')['href']

#------------------------article pdf-------------------------------------------------
pdf = article.find('div', class_='gs_or_ggsm').find('a')['href']



In [5]:
print("ID de l'article:", id)
print("Lien de l'article:", document_link)
print("Titre de l'article:", title_element)
print("Auteurs de l'article:")
for author in authors:
    print("- Nom:", author['name'])
    print("  Lien vers la biographie:", author['bio'])
print("Nombre de citations:", citation_count)
print("Lien des citations:", citation_link)
print("Nombre de versions:", version_count)
print("Lien des versions:", version_link)
print("Lien vers d'autres articles en relation:", autres_article)
print("Lien vers l'article PDF:", pdf)

ID de l'article: _hAUO1D456gJ
Lien de l'article: https://ieeexplore.ieee.org/abstract/document/8960342/
Titre de l'article: Starting MOOCs in African University: The experience of Cadi Ayyad University, process, review, recommendations, and prospects
Auteurs de l'article:
- Nom: K Berrada
  Lien vers la biographie: /citations?user=_3QgtDcAAAAJ&hl=fr&oi=sra
- Nom: R Bendaoud
  Lien vers la biographie: /citations?user=9rixKFEAAAAJ&hl=fr&oi=sra
- Nom: S Machwate
  Lien vers la biographie: /citations?user=01wHQ3IAAAAJ&hl=fr&oi=sra
Nombre de citations: 37
Lien des citations: /scholar?cites=12170969541464297726&as_sdt=2005&sciodt=0,5&hl=fr
Nombre de versions: 3
Lien des versions: /scholar?cluster=12170969541464297726&hl=fr&as_sdt=0,5
Lien vers d'autres articles en relation: /scholar?q=related:_hAUO1D456gJ:scholar.google.com/&scioq=university+cadi+ayyad&hl=fr&as_sdt=0,5
Lien vers l'article PDF: https://ieeexplore.ieee.org/iel7/6287639/8948470/08960342.pdf


**Fonction pour extarcter les infos necessaire d'aprés l'élémént html de l'article** 

In [6]:
import re

def scrape_mblock(article):
    article_info = {}

    try:
        # ID de l'article
        article_info['id'] = article.find('h3').find('a')['id']
    except:
        article_info['id'] = None

    try:
        # Lien de l'article
        article_info['document_link'] = article.find('h3').find('a')['href']
    except:
        article_info['document_link'] = None

    try:
        # Titre de l'article
        article_info['title'] = article.find('h3', {'class': 'gs_rt'}).text.strip()
    except:
        article_info['title'] = None

    try:
        # Auteurs de l'article
        authors_data = article.find('div', class_='gs_a').findAll('a')
        article_info['authors'] = []
        for author in authors_data:
            author_info = {
                'name': author.text,
                'bio': author['href']
            }
            article_info['authors'].append(author_info)
    except:
        article_info['authors'] = []

    try:
        # Nombre de citations de l'article
        citation_element = article.find(lambda tag: tag.name == 'a' and 'Cité' in tag.text)
        article_info['citation_count'] = int(re.search(r'\d+', citation_element.text).group())
        article_info['citation_link'] = citation_element['href']
    except:
        article_info['citation_count'] = None
        article_info['citation_link'] = None

    try:
        # Nombre de versions de l'article
        version_element = article.find(lambda tag: tag.name == 'a' and 'versions' in tag.text)
        article_info['version_count'] = int(re.search(r'\d+', version_element.text).group())
        article_info['version_link'] = version_element['href']
    except:
        article_info['version_count'] = None
        article_info['version_link'] = None

    try:
        # Lien vers d'autres articles en relation
        article_info['autres_articles_link'] = soup.find(lambda tag: tag.name == 'a' and tag.text == 'Autres articles')['href']
    except:
        article_info['autres_articles_link'] = None

    try:
        # Lien vers l'article PDF
        article_info['pdf_link'] = article.find('div', class_='gs_or_ggsm').find('a')['href']
    except:
        article_info['pdf_link'] = None

    return article_info

#**Scrapper les résultats d'une seul page de recherche** 

In [7]:
from bs4 import BeautifulSoup
def scrapping(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
    # url = 'https://scholar.google.com/scholar?hl=fr&as_sdt=0%2C5&q=university+cadi+ayyad&oq=university+cad'
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup


In [8]:
def scrape_m_page(article_blocks):
    
    page_article_data = []
    num_blocks = len(article_blocks)
    
    for block in range(num_blocks):
        page_article_data.append(scrape_mblock(article_blocks[block]))
    
    return page_article_data

Extract the next link : c'est à dire le lien de la page suivante

In [9]:

def extract_next_link(soup):
    try:
        next_link = soup.find('div', {'id': 'gs_n'})
        print(next_link)
        link=next_link.find_all('td')[-1].find('a')['href']
        link = "https://scholar.google.com/"+link
        return link
    except:
        return None 

    

Scrapper le nombres d'articles souhaité

In [11]:

def scrape(link, count):  
    i=0 
    data =[]
    while link is not None:
        soup = scrapping(link) 
        articles_blocks = soup.findAll('div', {'class': 'gs_r'})
        data.extend(scrape_m_page(articles_blocks)) 
        print(link)
        print(i)
        link = extract_next_link(soup)
       
        i=i+10
        if i == count:
            break
    return data
        

In [ ]:
link='https://scholar.google.com/scholar?hl=fr&as_sdt=0%2C5&q=university+cadi+ayyad&oq=university+cad'
data=scrape(link, 600)
len(data)


In [18]:
colonne_a_exporter = df['authors']

# Créer un nouveau DataFrame avec la colonne sélectionnée
nouveau_df = pd.DataFrame({'authors': colonne_a_exporter})

# Exporter le nouveau DataFrame vers un fichier Excel
nouveau_df.to_excel('export_excel.xlsx', index=False)

In [31]:
import pandas as pd
df=pd.DataFrame(data) 
pd.DataFrame(data)

,id,document_link,title,authors,citation_count,citation_link,version_count,version_link,autres_articles_link,pdf_link
0,_hAUO1D456gJ,https://ieeexplore.ieee.org/abstract/document/...,Starting MOOCs in African University: The expe...,"[{'name': 'K Berrada', 'bio': '/citations?user...",37.0,/scholar?cites=12170969541464297726&as_sdt=200...,3.0,/scholar?cluster=12170969541464297726&hl=fr&as...,/scholar?q=related:_hAUO1D456gJ:scholar.google...,https://ieeexplore.ieee.org/iel7/6287639/89484...
1,5Hl-3tWF9kUJ,https://books.google.com/books?hl=fr&lr=&id=rQ...,[LIVRE][B] … numérique: Actes du colloque annu...,"[{'name': 'L Gajo', 'bio': '/citations?user=E2...",11.0,/scholar?cites=5041363986495207908&as_sdt=2005...,2.0,/scholar?cluster=5041363986495207908&hl=fr&as_...,/scholar?q=related:_hAUO1D456gJ:scholar.google...,None
2,JVzuQCHZ0_gJ,https://agritrop.cirad.fr/508400/,Modélisation de l'architecture du palmier datt...,"[{'name': 'MA Elhoumaizi', 'bio': '/citations?...",23.0,/scholar?cites=17929913278338587685&as_sdt=200...,3.0,/scholar?cluster=17929913278338587685&hl=fr&as...,/scholar?q=related:_hAUO1D456gJ:scholar.google...,None
3,veNVWfpTQd0J,https://link.springer.com/chapter/10.1007/978-...,Agility of the post COVID-19 strategic plan on...,"[{'name': 'K El Kharki', 'bio': '/citations?us...",18.0,/scholar?cites=15943116490573996989&as_sdt=200...,NaN,None,/scholar?q=related:_hAUO1D456gJ:scholar.google...,None
4,uivW1Psh50wJ,https://access.archive-ouverte.unige.ch/access...,"[PDF][PDF] Articuler présence et distance, une...","[{'name': 'B Charlier', 'bio': '/citations?use...",20.0,/scholar?cites=5541435231993998266&as_sdt=2005...,3.0,/scholar?cluster=5541435231993998266&hl=fr&as_...,/scholar?q=related:_hAUO1D456gJ:scholar.google...,https://access.archive-ouverte.unige.ch/access...
...,...,...,...,...,...,...,...,...,...,...
655,nQLx_LRrvFAJ,https://theses.hal.science/tel-02085104/,Tangential methods for model reductions and ap...,"[{'name': 'Y Kaouane', 'bio': '/citations?user...",2.0,/scholar?cites=5817643243737776797&as_sdt=2005...,3.0,/scholar?cluster=5817643243737776797&hl=fr&as_...,/scholar?q=related:_hAUO1D456gJ:scholar.google...,https://theses.hal.science/tel-02085104/file/T...
656,Z_-6Jop4BGUJ,https://www.igi-global.com/chapter/therapeutic...,Therapeutic strategies for treatment of COVID-19,"[{'name': 'L Tamegart', 'bio': '/citations?use...",1.0,/scholar?cites=7279075432488173415&as_sdt=2005...,2.0,/scholar?cluster=7279075432488173415&hl=fr&as_...,/scholar?q=related:_hAUO1D456gJ:scholar.google...,None
657,Ty2JkR3igtQJ,https://ej-med.org/index.php/ejmed/article/vie...,Bilateral Sphenopalatine Artery Embolization i...,[],NaN,None,2.0,/scholar?cluster=15313050299636723023&hl=fr&as...,/scholar?q=related:_hAUO1D456gJ:scholar.google...,https://ej-med.org/index.php/ejmed/article/dow...
658,-O66_Rxx0OkJ,https://www.inderscienceonline.com/doi/abs/10....,Resource allocation algorithm for multi-user M...,"[{'name': 'H Ayad', 'bio': '/citations?user=bD...",1.0,/scholar?cites=16848090575322935032&as_sdt=200...,4.0,/scholar?cluster=16848090575322935032&hl=fr&as...,/scholar?q=related:_hAUO1D456gJ:scholar.google...,None


isoler la colonne authors dans un autre dataframe

In [ ]:
import pandas as pd


# Créez une liste vide pour stocker les nouveaux enregistrements
new_records = []

# Parcourez chaque ligne de la colonne "authors"
for authors_list in df['authors']:
    # Initialisez une liste temporaire pour stocker les nouveaux enregistrements de chaque ligne
    temp_records = []
    
    # Parcourez chaque dictionnaire dans la liste
    for author_dict in authors_list:
        # Vérifiez si le dictionnaire n'est pas vide
        if author_dict:
            # Ajoutez le dictionnaire à la liste temporaire
            temp_records.append({
                'name': author_dict['name'],
                'bio': f"https://scholar.google.com{author_dict['bio']}"  # Ajoutez le préfixe de l'URL
            })
    
    # Ajoutez la liste temporaire à la liste des nouveaux enregistrements
    new_records.extend(temp_records)

# Créez un nouveau DataFrame à partir de la liste des nouveaux enregistrements
new_df = pd.DataFrame(new_records)



On supprime les doublons 

In [ ]:
# Supprimez les doublons basés sur la colonne "name"
new_df_no_duplicates = new_df.drop_duplicates(subset='name')

# Affichez le nouveau DataFrame sans doublons
pd.Dataframe(new_df_no_duplicates)


In [32]:
pd.DataFrame(new_df_no_duplicates)

,name,bio
0,K Berrada,https://scholar.google.com/citations?user=_3Qg...
1,R Bendaoud,https://scholar.google.com/citations?user=9rix...
2,S Machwate,https://scholar.google.com/citations?user=01wH...
3,L Gajo,https://scholar.google.com/citations?user=E239...
4,V Larivière,https://scholar.google.com/citations?user=UvfK...
...,...,...
505,R Abderrahmane,https://scholar.google.com/citations?user=Dfgd...
507,Y Kaouane,https://scholar.google.com/citations?user=JUJ2...
508,L Tamegart,https://scholar.google.com/citations?user=2OXK...
509,H Ayad,https://scholar.google.com/citations?user=bD5-...


On sauvgarde les résulats sous forme de fichier xlsx

In [ ]:
new_df_no_duplicates.to_excel('resultat.xlsx', index=False)